# Welcome  

Notebook Author: Samuel Alter  
Notebook Subject: Capstone Project - Preprocess Imagery

BrainStation Winter 2023: Data Science

This notebook is for processing the satellite into chunks for modelling.

In [1]:
# imports

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

## Clip raster to polygon

In [ ]:
from osgeo import gdal, ogr

# Define the input raster and polygon mask
input_raster = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/m_3411849_se_11_060_20180722.tif"
mask = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_fire_valid.shp"

# Open the input raster and polygon mask
raster_ds = gdal.Open(input_raster)
mask_ds = ogr.Open(mask)

# Get the mask layer
mask_lyr = mask_ds.GetLayer()

# Get the extent of the mask layer
mask_extent = mask_lyr.GetExtent()

# Set the output file name and format
output_file = "path/to/clipped_raster.tif"
output_format = "GTiff"

# Set the output file resolution
output_res = raster_ds.GetGeoTransform()[1]

# Define the output file size
output_width = int((mask_extent[1] - mask_extent[0]) / output_res)
output_height = int((mask_extent[3] - mask_extent[2]) / output_res)

# Define the warp options
warp_options = gdal.WarpOptions(cutlineDSName=mask, cropToCutline=True, dstSRS=raster_ds.GetProjection(), outputBounds=mask_extent, xRes=output_res, yRes=output_res, width=output_width, height=output_height)

# Call the gdal.Warp() function to clip the raster
clipped_raster_ds = gdal.Warp(output_file, raster_ds, options=warp_options)

# Save clipped raster to a shapefile
output_shp = "path/to/clipped_raster.shp"
gdal.VectorTranslate(output_shp, clipped_raster_ds, format="ESRI Shapefile")

# Save clipped raster to a GeoJSON
output_geojson = "path/to/clipped_raster.geojson"
gdal.VectorTranslate(output_geojson, clipped_raster_ds, format="GeoJSON")

# Clean up
raster_ds = None
mask_ds = None
clipped_raster_ds = None

In [ ]:
from osgeo import gdal, ogr

# Define the input raster and polygon mask
input_raster = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/m_3411849_se_11_060_20180722.tif"
mask = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_fire_valid.shp"

# Open the input raster and polygon mask
raster_ds = gdal.Open(input_raster)
mask_ds = ogr.Open(mask)

# Get the mask layer
mask_lyr = mask_ds.GetLayer()

# Get the extent of the mask layer
mask_extent = mask_lyr.GetExtent()

# Set the output file name and format for GeoTIFF
output_file_tif = "path/to/clipped_raster.tif"
output_format_tif = "GTiff"

# Set the output file name and format for GeoJSON
output_file_geojson = "path/to/clipped_raster.geojson"
output_format_geojson = "GeoJSON"

# Set the output file resolution
output_res = raster_ds.GetGeoTransform()[1]

# Define the output file size
output_width = int((mask_extent[1] - mask_extent[0]) / output_res)
output_height = int((mask_extent[3] - mask_extent[2]) / output_res)

# Define the warp options
warp_options = gdal.WarpOptions(cutlineDSName=mask, cropToCutline=True, dstSRS=raster_ds.GetProjection(), outputBounds=mask_extent, xRes=output_res, yRes=output_res, width=output_width, height=output_height)

# Call the gdal.Warp() function to clip the raster
clipped_raster_ds = gdal.Warp('', raster_ds, options=warp_options)

# Save clipped raster to GeoTIFF
output_tif = "path/to/clipped_raster.tif"
gdal.Translate(output_tif, clipped_raster_ds, format=output_format_tif)

# Save clipped raster to GeoJSON
output_geojson = "path/to/clipped_raster.geojson"
gdal.Translate(output_file_geojson, clipped_raster_ds, format=output_format_geojson)

# Save clipped raster to a shapefile
output_shp = "path/to/clipped_raster.shp"
gdal.VectorTranslate(output_shp, clipped_raster_ds, format="ESRI Shapefile")

# Clean up
raster_ds = None
mask_ds = None
clipped_raster_ds = None

## [Extrating Patches from Large Images ~~and Masks~~ for Semantic Segmentation](https://www.youtube.com/watch?v=7IL7LKSLb9I)

Following this tutorial to convert my large fire/nofire images into patches for neural network analysis. The code block below is from this video, with some alterations to adapt it to my use case.

In [10]:
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify
import tifffile as tiff

# large_image_stack_fire=tiff.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_fire.tif')

large_image_stack_patch_fire=tiff.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/m_3411849_se_11_060_20180722/m_3411849_se_11_060_20180722_fire.tif')
# large_image_stack_patch_fire

In [18]:
# updated 
# https://stackoverflow.com/questions/68224588/problem-when-using-patchify-library-to-create-patches

import cv2

# filepaths
target_tiff_fire=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/m_3411849_se_11_060_20180722/m_3411849_se_11_060_20180722_converted_fire.tif'
output_location_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/m_3411849_se_11_060_20180722/patches_fire'

# read large_image_stack_test
img = cv2.imread(target_tiff_fire)

cv2.imshow('image',img)

error: OpenCV(4.6.0) /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/imgcodecs/src/loadsave.cpp:77: error: (-215:Assertion failed) pixels <= CV_IO_MAX_IMAGE_PIXELS in function 'validateInputImageSize'


In [3]:
patches_img = patchify(img, (128,128,3), step=128)

for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        single_patch_img = patches_img[i, j, 0, :, :, :]
        if not cv2.imwrite(output_location_fire + 'image_' + '_'+ str(i)+str(j)+'.jpg', single_patch_img):
            raise Exception("Could not write the image")

error: OpenCV(4.6.0) /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/imgcodecs/src/loadsave.cpp:77: error: (-215:Assertion failed) pixels <= CV_IO_MAX_IMAGE_PIXELS in function 'validateInputImageSize'
